Natural Language Processing
Review Data from Airbnb

In [82]:
#Read in libraries
import pandas as pd
import swifter
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import sklearn

In [83]:
#Settings for notebook
#Ignore warnings
import warnings; warnings.simplefilter('ignore')

#Increase number of columns and rows displayed by Pandas
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

In [84]:
#Set path to data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\03_Processed'

#Read in data
reviews = pd.read_csv(path + '/01_10_2020_Reviews_Processed_Text_Analysis.csv',sep=',',
                 parse_dates=['date'])

In [85]:
reviews.head().T

,0,1,2,3,4
comments,This a jewel in the DogpatchPotrero area Super easy and prompt communication If driving easy access to the 101 and 280 as an added bonus Much recommended,Megs place is sparkling clean and in an awesome location Meg is also very friendly and so does the dog shes so sweet,We had a great time in SF and loved staying at this home Marjorie was most helpful and informative about getting around and using public transportation It was very easy and we didnt feel far from all the city attractions The accommodation was comfortable and very nice I would stay there again and recommend it to friends and family Thanks so much,Very cozy place with the beach literally right across the street Very clean Would definitely stay again,Kevin is very professional prompt and warmhearted individual He gave us a lot of good tips traveling the city Definitely five stars
date,2016-01-26 00:00:00,2019-03-08 00:00:00,2015-08-16 00:00:00,2018-10-17 00:00:00,2018-01-10 00:00:00
id_review,60697388,421167935,42860595,337874233,226453615
listing_id,288213,10427768,4269254,4252808,1393654
reviewer_id,308818,32888504,35391497,76550363,133118406
reviewer_name,Desigan,家惠,Hallie,Patrick,Daniel
host_is_superhost,True,True,True,True,True
host_response_time,within an hour,within an hour,within an hour,within an hour,within an hour
latitude,37.7597,37.7736,37.783,37.7573,37.7524
longitude,-122.388,-122.426,-122.432,-122.509,-122.459


Histogram of sentiment scores

In [ ]:
#Set style 
plt.style.use('ggplot')

#Set plot
fig, ax = plt.subplots(2,2, figsize = (10,10))

fig.suptitle('Sentiment Scores for SF Airbnb Reviews',fontsize =19,fontweight = 'bold', y =.95)
fig.text(.45,.07, 'Sentiment Scores', fontweight='bold')
fig.text(.05,.45, 'Number of Reviews', fontweight='bold', rotation = 'vertical')

#Plot 4 histograms
ax1 = reviews.sentiment_neg.hist(ax = ax[0,0],bins = 20, color = '#5f6caf', alpha = .75)
ax1.set_title("Negative")

ax2 = reviews.sentiment_pos.hist(ax = ax[0,1],bins = 20, color = '#7fcd91',alpha = .75)
ax2.set_title("Positive")

ax3 = reviews.sentiment_neu.hist(ax = ax[1,0],bins = 20, color = '#0f4c75',alpha = .5)
ax3.set_title("Neutral")

ax4 = reviews.sentiment_compound.hist(ax = ax[1,1],bins = 20, color = '#1b262c',alpha = .5)
ax4.set_title("Compound")


Text(0.5, 1.0, 'Compound')

In [ ]:
#Statistics in reference to sentiment scores
percentiles = reviews['sentiment_compound'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
percentiles

In [ ]:
#split sentiment_compound scores into 20 bins
reviews['binned'] = pd.cut(reviews.sentiment_compound, bins = 20)

#Set fig size
fig, ax = plt.subplots(figsize = (10,5))

#Plot data
ax = sns.countplot(y = 'binned',data = reviews, ax=ax, color = 'darkgreen')




Our reviews are very positive. 
Let's compare the differences between the very positive comments and the very negative ones

## Differences in review length

In [ ]:
reviews['comments_length'] = reviews.comments_parsed.apply(len)

positive_reviews = reviews[reviews.sentiment_compound > .99]
negative_reviews = reviews[reviews.sentiment_compound < -.5]

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

#Plot Histogram
sns.distplot(positive_reviews.comments_length, kde = True, bins = 50, color = 'b')
sns.distplot(negative_reviews.comments_length, kde = True, bins = 50, color = 'r')

plt.legend(labels = ['Positive Reviews', 'Negative Reviews'], frameon = True);
ax.set_title('Airbnb Review Length Distribution')
ax.set_xlabel('Characters')

Let's look at a subset of some of these reviews

Postive Reviews

In [ ]:
#Positive reviews
positive_reviews.comments[10:15]

Word Clouds


In [ ]:
#Positive Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image


In [ ]:
wordcloud = WordCloud(max_font_size=200, max_words=200, background_color="skyblue",
                      width= 3000, height = 2000).generate(str(positive_reviews.comments.values))

def plot_wordcloud(wordcloud, language):
    plt.figure(figsize=(12, 10))
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis("off")
    plt.title(language + ' Comments\n', fontsize=18, fontweight='bold')
    plt.show()
    
plot_wordcloud(wordcloud, '\nPositively Tuned')

In [ ]:
#What are the most popular words in positive reviews?

In [ ]:
#Import count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english').fit(positive_reviews.comments)

print('Vocabulary used in positive reviews: {}'.format(len(count.vocabulary_)))

bow = count.transform(positive_reviews.comments)
sum_words = bow.sum(axis=0)

#we sort a list of tuples that contain the word and their occurrence in the corpus
words_freq = [(word, sum_words[0, idx]) for word, idx in count.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [ ]:
positive_words = pd.DataFrame(words_freq, columns =['Word', 'Count']) 


In [ ]:
positive_words = positive_words.head(30)

fig,ax = plt.subplots(figsize = (10,10))
positive_words.sort_values(by='Count').plot(x = 'Word',y='Count',kind='barh', ax=ax)

ax.set_title('Most Common Words used in Positive Airbnb Reviews')
ax.set_ylabel('')

In [ ]:
#Topic modeling and document clustering for postive reviews

In [ ]:
#Import libraries
import gensim
from gensim.parsing.preprocessing import preprocess_string
from gensim.models import Phrases
from gensim import corpora, models, similarities

In [ ]:
#Apply preprocess and stem original comments
docs = np.array(positive_reviews['comments'].apply(preprocess_string))

# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
#Create dictionary of tokens and print info
dictionary = corpora.Dictionary(docs)
print('Number of unique words in original documents: {}'.format(len(dictionary)))

**Remove rare or common tokens**

In [ ]:
#Filter words that occur in less than 10 documents or more than 25% of documents
dictionary.filter_extremes(no_below= 10,no_above=.25 )

print('Number of unique words after removing rare and common words: {}'.format(len(dictionary)))

In [ ]:
#Vectorize data
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
#Initialize Tfidf and fit to corpus
tfidf = models.TfidfModel(corpus)

#Scale and transform corpus
transformed_tfidf = tfidf[corpus]

#Fit transformed data to Lda
lda = models.LdaMulticore(corpus= transformed_tfidf, num_topics=10, id2word=dictionary)

In [ ]:
#Import pyLDAvis to visualize topics found in LDA
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#Create plot
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

**Takeaways**

## Negative Reviews

In [ ]:
#Preview some of the negative reviews
negative_reviews.comments[10:15]

**Wordcloud of negative comments**

In [ ]:
#Create wordcloud
wordcloud = WordCloud(max_font_size=200, max_words=200, background_color="lightgray",
                      width= 3000, height = 2000).generate(str(negative_reviews.comments.values))

#Plot wordcloud
plot_wordcloud(wordcloud, '\nNegatively Tuned')

#What are the most popular words in negative reviews?

In [ ]:
#Instantiate CountVectorizer and fit to comments
count = CountVectorizer(stop_words='english').fit(negative_reviews.comments)

#Print length of unique words in negative comments
print('Vocabulary used in negative reviews: {}'.format(len(count.vocabulary_)))

#Create bag of words from comments
bow = count.transform(negative_reviews.comments)
sum_words = bow.sum(axis=0)

#we sort a list of tuples that contain the word and their occurrence in the corpus
words_freq = [(word, sum_words[0, idx]) for word, idx in count.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [ ]:
#Create df with vocab and their corresponding counts 
negative_words = pd.DataFrame(words_freq, columns =['Word', 'Count']) 

In [ ]:
#Capture the 30 most used words
negative_words = negative_words.head(30)

#Plot
fig,ax = plt.subplots(figsize = (10,10))
negative_words.sort_values(by='Count').plot(x = 'Word',y='Count',kind='barh', ax=ax, color = 'skyblue')

#Set plot aesthetics
ax.set_title('Most Common Words used in Negative Airbnb Reviews')
ax.set_ylabel('')

In [ ]:
#Topic modeling for negative reviews

In [ ]:
#Apply preprocess and stem original comments
docs = np.array(negative_reviews['comments'].apply(preprocess_string))

# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
#Create dictionary of tokens and print info
dictionary = corpora.Dictionary(docs)
print('Number of unique words in original documents: {}'.format(len(dictionary)))

**Remove rare or common tokens**

In [ ]:
#Filter words that occur in less than 10 documents or more than 25% of documents
dictionary.filter_extremes(no_below= 10,no_above=.25 )

print('Number of unique words after removing rare and common words: {}'.format(len(dictionary)))

In [ ]:
#Vectorize data
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
#Initialize Tfidf and fit to corpus
tfidf = models.TfidfModel(corpus)

#Scale and transform corpus
transformed_tfidf = tfidf[corpus]

#Fit transformed data to Lda
lda = models.LdaMulticore(corpus= transformed_tfidf, num_topics=10, id2word=dictionary)

In [ ]:
#Create plot
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

In [ ]:
#what insights can you gain?